# Writing unit tests
<img src='../images/xebia-logo.png' width='300px' align='right' style="padding: 15px">

This notebook will walk you through how to write unit tests with `pytest`.

The first step is installing `pytest` with: 

```bash
uv add --dev pytest
```

There are multiple [test discovery rules](https://docs.pytest.org/en/7.1.x/explanation/goodpractices.html#test-discovery), which is what enables pytest to identify where the test scripts are located. 

The TL;DR is: When no [testpaths](https://docs.pytest.org/en/7.1.x/reference/reference.html#confval-testpaths) or path arguments in the CLI command are specified, pytest goes through each folder and looks for files starting or ending with the substring `test_*` or `*_test`.

In general terms, you should mimic the structure of your package in your `tests/` folder, prepending each file with `test_*.py` and each testing function or class with `test_*`, too.

Your test folder structure could look like this:

```plain
├── src/​
│  └─ animal_shelter/​
│     ├── __init__.py​
│     ├── data.py​
│     └── features.py​
└── tests/​
│   ├── test_data.py​
│   └── test_features.py​
└── pyproject.toml​
```

#### <mark> Exercise:</mark> The first test

For example, to test the `convert_camel_case*()` function from `data.py`, you would create the file `tests/test_data.py` with the following contents:

In [ ]:
from animal_shelter import data

def test_convert_camel_case():
    assert data.convert_camel_case("CamelCase") == "camel_case"
    assert data.convert_camel_case("CamelCASE") == "camel_case"
    assert data.convert_camel_case("camel-case") != "camel_case"

**Try running `pytest​` from the virtual environment.**

The `assert` keyword defines a expression that expects a boolean. If the boolean is `True` the program continues, and if it's `False` it raises an exception, which `pytest` interprests as the test not passing.

**Try adding extra `assert` statements.**
For example,
- What would you expect `conver_camel_case()` to return if the input is already in *snake_case*?
- What if the input contains whitespace?

## Other assertions

The `assert` keyword is very flexible and is all the syntaxt you would need to write a lot of tests, but there are some other assertion-like functions that come in handy.

For example the `assert_series_equal()` from the `pandas.testing` module allows to easily compare that the values of two `Series` are the same.

#### <mark> Exercise: </mark> The second test

Can you add the following test for the `check_has_name()` function from `features.py`?

Think about in which file you need to add it.

In [ ]:
from pandas.testing import assert_series_equal

from animal_shelter import features

def test_check_has_name():
    s = pd.Series(["Ivo", "Henk", "unknown"])
    result = features.check_has_name(s)
    expected = pd.Series([True, True, False])
    assert_series_equal(result, expected)

#### <mark> Exercise: </mark> The third, fourth.... test

Add now at least one unit test for each function in `features.py` using `assert_series_equal()`.

### Checking for exceptions

Another common kind of assertion, for which you might want to test, is checking that something doesn't work when you expect it not to.
In the simplest case, you can use an inequality logical comparator with an `assert` statement (e.g. `assert x != y`).
But sometimes, you want to check that the calling function actually throws an error.
To check for exceptions you can use the `pytest.raises()` context manager.

```python
import pytest

def test_for_exceptions():
    with pytest.raises(EXCEPTION_TYPE):
        function_that_errors()
```

#### <mark> Exercise:</mark> Checking for exceptions 

- Add a test that checks that `convert_camel_case()` throws an exception when called with a value that is not a `str`. Check what kind of exception the function raises in that case.
- Can you also add some exception checking to some of the functions in `features.py`?

When checking for exceptions is important that the checks are as precise as possible, so that your tests don't pass when *any* error occurs, but only the error you were expecting. To do so, you can write explicit assertions using the exception object that is produced. If you wonder what kind of exceptions are built into Python already, you can have a look at the official Python [documentation](https://docs.python.org/3/library/exceptions.html).

```python
import pytest

def test_for_exceptions():
    with pytest.raises(EXCEPTION_TYPE) as exception:
        function_that_errors()
    assert "exception message" in str(exception.value)
```


### A note on testing

It can be hard to see the value of testing when doing this kind of exercises, but the practice of writing *real* tests is very different than trying to test a code base post-hoc. During the development process, you usually write multiple attempts of each function you write, and those attempts can give you insight into which tests are likely to be more valuable and catch potential issues in the future. 

Writing a meaningful testing suite is usually an organic process and which tests are good to check for comes from experience working on a project.

## Fixtures

Fixtures allow you to define functions that setup elements required by (multiple) tests.

For example, if two tests use the same input (data), you can put it into a fixture. To define fixtures, you can decorate a function with the `@pytest.fixture` decorator. When you call a testing function that accepts arguments, `pytest` will check if there are fixtures with the same argument names and automatically pass them to the testing functions.



In [ ]:
import pytest

@pytest.fixture()
def list_of_numbers():
    return [1, 2, 3, 4, 5]

def test_all_nums(list_of_numbers):
    assert all(type(element) is int for element in list_of_numbers)

def test_sum(list_of_numbers):
    assert sum(list_of_numbers) == 15

The example above behaves simmilarly as having a single testing function with two `assert` expressions.

However, fixtures are very flexible and their utility shines through with more complex usecases.

The `@fixture` decorator accepts an argument called `scope` that determines *the lifetime of fixtures*. By default `scope="function"`, which means that each function that requires a fixture gets a new copy of the output of the fixture. 

Changing this parameter is useful if, for example, initializing the inputs required for testing is expensive computationally (e.g. connecting to a database). We can choose to group all tests that require the same input into a `class` and set the fixture `scope="class"`. In this case `list_of_numbers` will be generated only once, and passed to both tests.

**Warning:** If one of the tests were to mutate `list_of_numbers`, that mutation would carry to the next tests.

In [ ]:
import pytest

@pytest.fixture(scope="class")
def list_of_numbers():
    return [1, 2, 3, 4, 5]

class TestListFunctions:
    def test_all_nums(self, list_of_numbers):
        assert all(type(element) is int for element in list_of_numbers)

    def test_sum(self, list_of_numbers):
        assert sum(list_of_numbers) == 15

Another common fixture scope is `"module"`, where the fixture object is the same for all testing functions within a module (i.e. `.py` file).

#### <mark> Exercise:</mark> Add a fixture

On `features.py`, there are two functions that generate features from the `sex_upon_outcome` variable of the data.

Can you write unit tests for them (or re-use the ones you wrote in the previous exercises) that use a fixture to provide the same *mocked* input data for both tests?

## Other fixture facts
Once you get used to working with fixtures, they usually offer a more ergonomic and modular way of designing your testing suits than writing a lot of abstractions yourself.

Let's look at another few things you can do with them:

#### Fixtures can depend on fixtures

In [ ]:
@pytest.fixture()
def list_of_numbers():
    return [1, 2, 3]

@pytest.fixture()
def list_of_more_numbers(list_of_numbers):
    return list_of_numbers + [4, 5]

def test_sum(list_of_more__numbers):
    assert sum(list_of_more_numbers) == 15

#### Testing functions can depend on multiple fixtures

In [ ]:
@pytest.fixture()
def list_of_numbers():
    return [1, 2, 3]

@pytest.fixture()
def more_numbers():
    return [4, 5]

def test_sum(list_of_numbers, more_numbers):
    assert sum(list_of_more_numbers) + sum(more_numbers) == 15

# Pytest with pre-commit

You can add `pytest` to `pre-commit` by adding a *local* hook to your `.pre-commit-config.yaml`.

```yaml
- repo: local
  hooks:
    - id: pytest
      name: pytest
      entry: pytest
      language: system
      pass_filenames: false
      always_run: true
```

However, running tests automatically before commits go through might be too intrusive, so be mindful of the trade-offs of installing this hook.